# WDI Data

##### Datele au fost culese pe toate tarile din Africa, urmarind 5 indicatori legati de nivelul de guvernanta pe care tarile il au. Am urmarit ultimii 25 de ani pentru aceasta tema si vom corobora aceste detalii cu date luate de pe EIA legate de domeniul electricitatii.

##### Am ales acesti indicatori cu ipoteza ca un stat care are o proasta guvernare (conform indicatorilor alesi), ar trebui sa aiba si un management prost al resurselor electrice, dar in egala masura sunt si curios daca exista o relatie de interdependenta intre aceste doua idei, in sensul in care si domeniul electric prost poate influenta cum sunt guvernati oamenii (mai bine, cu mai putina coruptie, fara regim miliat, in standarde democratice, etc.).

##### Ca disclaimer, WDI nu ofera nicio data pentru 2001, si nici pentru 2024 si 2025. Ultimul indicator numit SI.POV.GINI este completat foarte sporadic, incat s-ar putea sa il scoatem pe de-a-ntregul din analiza in momentul in care vom uni datele si cu EIA Data

## Configuratii

In [1]:
import wbgapi as wb
import pandas as pd

WDI_INDICATORS = {
    "gov_effectiveness": "GE.EST",
    "regulatory_quality": "RQ.EST",
    "rule_of_law": "RL.EST",
    "control_corruption": "CC.EST",
    "gini_coefficient": "SI.POV.GINI",
}
YEARS = range(2000, 2024)
output_filename = "african_wdi_governance_data_2000_2023_complete.csv"

AFRICA_COUNTRY_CODES = [
    'DZA', 'AGO', 'BEN', 'BWA', 'BFA', 'BDI', 'CPV', 'CMR', 'CAF', 'TCD', 'COM', 
    'COG', 'CIV', 'COD', 'ERI', 'ETH', 'GAB', 'GMB', 'GHA', 'GIN', 'GNB', 'GNQ', 
    'KEN', 'LSO', 'LBR', 'LBY', 'MDG', 'MWI', 'MLI', 'MRT', 'MUS', 'MAR', 'MOZ', 
    'NAM', 'NER', 'NGA', 'RWA', 'STP', 'SEN', 'SYC', 'SLE', 'SOM', 'ZAF', 'SSD', 
    'SDN', 'SWZ', 'TGO', 'TUN', 'UGA', 'TZA', 'ZMB', 'ZWE', 'EGY', 'DJI'
]

## Luam datele

In [2]:
try:
    df_raw = wb.data.DataFrame(
        series=list(WDI_INDICATORS.values()),
        economy=AFRICA_COUNTRY_CODES,
        time=YEARS,
        labels=True,
        skipAggs=True
    )
    
    print(f"Raw DataFrame Shape: {df_raw.shape}")
    print("\nRaw Data Head (Indexed by Country Code and WDI Code, Years are columns):")
    print(df_raw.head())
    
except Exception as e:
    print(f"An error occurred while fetching data: {e}")
    df_raw = pd.DataFrame()

Raw DataFrame Shape: (270, 26)

Raw Data Head (Indexed by Country Code and WDI Code, Years are columns):
                         Country                              Series  \
economy series                                                         
DJI     GE.EST          Djibouti  Government Effectiveness: Estimate   
EGY     GE.EST  Egypt, Arab Rep.  Government Effectiveness: Estimate   
ZWE     GE.EST          Zimbabwe  Government Effectiveness: Estimate   
ZMB     GE.EST            Zambia  Government Effectiveness: Estimate   
TZA     GE.EST          Tanzania  Government Effectiveness: Estimate   

                  YR2000  YR2001    YR2002    YR2003    YR2004    YR2005  \
economy series                                                             
DJI     GE.EST -0.904506     NaN -0.764728 -0.710450 -0.635513 -0.816438   
EGY     GE.EST -0.174759     NaN -0.446098 -0.389789 -0.278711 -0.399057   
ZWE     GE.EST -0.757243     NaN -0.839923 -0.935436 -1.007126 -1.358318   
ZMB     GE

## Curatam putin datele

##### Am decis sa pastrez si valorile care nu au date (adica anul 2001 pentru a fi mai usor de unit cu EIA). Daca se va decide sa taiem putin din perioada pe care o analizam, va fi vocea poporului care a confirmat.

#### SI.POV.GINI l-am pastrat ca, in urma unei decizii cu echipa, sa vedem daca il pastram sau nu.

In [3]:
if not df_raw.empty:
    df = df_raw.reset_index()
    id_vars = ['economy', 'series', 'Country', 'Series'] 
    value_vars = [col for col in df.columns if col.startswith('YR')]
    
    if not value_vars:
        print("Error: Could not identify year columns (starting with 'YR').")
        df_final = pd.DataFrame() 
    else:
        df_long = df.melt(
            id_vars=id_vars,
            value_vars=value_vars,
            var_name='Year_Code',
            value_name='Value'
        ) 

        df_long['Year'] = df_long['Year_Code'].str.replace('YR', '').astype(int)
        
        indicator_rename_map = {v: k for k, v in WDI_INDICATORS.items()}

        df_final = df_long.rename(columns={
            'economy': 'Country_Code', 
            'Country': 'Country_Name', 
            'series': 'Indicator_Code'
        }).drop(columns=['Year_Code', 'Series']) 

        df_final['Indicator_Name'] = df_final['Indicator_Code'].map(indicator_rename_map)
        
        df_final = df_final[['Country_Name', 'Country_Code', 'Year', 'Indicator_Name', 'Indicator_Code', 'Value']]

    print(f"Final Row Count: {len(df_final)}")

Final Row Count: 6480


## Salvam totul in CSV

In [6]:

if 'df_final' in locals() and not df_final.empty:
    df_final.to_csv(output_filename, index=False)
    print(f"The data has been saved to: **{output_filename}**")

The data has been saved to: **african_wdi_governance_data_2000_2023_complete.csv**
